# Total swaps

This notebook shows a method to get Bancor trade volumes over a time period.

Set the blocks and sum the volume to get 24h volume, daily trading volume, etc.

This is a suitable method over larger timeframes.


## Setup
---

In [1]:
from graphqlclient import GraphQLClient
import pandas as pd
import json
from pandas.io.json import json_normalize
from datetime import datetime

In [2]:
ENDPOINT = "https://api.thegraph.com/subgraphs/name/blocklytics/bancor"
client = GraphQLClient(ENDPOINT)

This examples takes ~24 hour period by manually selecting blocks. In practice, this should be automated for the relevant periods.

In [3]:
# Blocks
start_block = 9273388 # Inclusive
end_block = 9279958   # Inclusive?

## Fetch data
---

In [4]:
# This function accepts a query starting with 'results'
# Returns a dataframe with flattened results

def fetch_results(query, block):
    # Start with an empty df
    df = pd.DataFrame()

    # Results must be paginated
    limit = 100
    offset = 0
    fetching_results = True
    # Fetch paginated results
    while fetching_results:
        _query = query.format(limit, offset, block)
        
        result = json.loads(client.execute(_query))
        _df = pd.DataFrame.from_records(json_normalize(result['data']['results']))

        # If df is empty, create it from new _df
        # Else append new _df to df
        if len(df.index) == 0:
            df = _df
        else:
            df = df.append(_df)

        # Prepare for pagination
        result_length = len(_df.index)
        if limit == result_length:
            offset += limit
        else:
            fetching_results = False

    df.reset_index(drop=True, inplace=True)
    print(df.shape)
    return df

In [16]:
query = """
    {{
      results:tokenSwapTotals(
        first:{0},
        skip:{1},
        block:{{number:{2}}}
        )
        {{
        fromToken {{id, symbol, decimals}}
        toToken {{id, symbol, decimals}}
        totalAmountPurchased
        totalAmountReturned
      }}
    }}
"""

df_start = fetch_results(query, start_block)
df_end = fetch_results(query, end_block)

(467, 8)
(467, 8)


# Format data
---

Combine dataframes

In [6]:
df = df_start\
    .merge(df_end, 
           how='outer',
           on=['fromToken.decimals',  'fromToken.id', 'fromToken.symbol', 'toToken.decimals', 'toToken.id', 'toToken.symbol'],
           suffixes=('_start', '_end')
          )

Convert from wei

In [7]:
df['totalAmountPurchased_start_f'] = df.apply(lambda row: float(row['totalAmountPurchased_start']) / 10 ** row['fromToken.decimals'], axis=1)
df['totalAmountPurchased_end_f'] = df.apply(lambda row: float(row['totalAmountPurchased_end']) / 10 ** row['fromToken.decimals'], axis=1)

df['totalAmountReturned_start_f'] = df.apply(lambda row: float(row['totalAmountReturned_start']) / 10 ** row['toToken.decimals'], axis=1)
df['totalAmountReturned_end_f'] = df.apply(lambda row: float(row['totalAmountReturned_end']) / 10 ** row['toToken.decimals'], axis=1)


Get difference over time period

In [8]:
df['totalAmountPurchased'] = df['totalAmountPurchased_end_f'] - df['totalAmountPurchased_start_f']
df['totalAmountReturned'] = df['totalAmountReturned_end_f'] - df['totalAmountReturned_start_f']

Get volume in base token terms (USDB or BNT)

In [18]:
df['volume_bnt'] = \
    df.apply(lambda row: row['totalAmountPurchased'] if row['fromToken.id'] == "0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c" else 0, axis=1)\
    + df.apply(lambda row: row['totalAmountReturned'] if row['toToken.id'] == "0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c" else 0, axis=1)

df['volume_usdb'] = \
    df.apply(lambda row: row['totalAmountPurchased'] if row['fromToken.id'] == "0x309627af60f0926daa6041b8279484312f2bf060" else 0, axis=1)\
    + df.apply(lambda row: row['totalAmountReturned'] if row['toToken.id'] == "0x309627af60f0926daa6041b8279484312f2bf060" else 0, axis=1)\


# Volumes over period
---
## BNT Volume

In [19]:
df['volume_bnt'].sum()

1301370.1838064648

## USDB Volume

Avoid double-counting volume by excluding BNT volume already counted above.

In [20]:
# df['volume_usdb'].sum() # This will double-count BNT volume.

In [21]:
df[df['volume_bnt'] == 0]['volume_usdb'].sum()

2099.013373769474

## Trade pairs not counted

These trade pairs were not counted as they did not include a base token of BNT or USDB.

In [22]:
df[(df['volume_bnt'] == 0) & (df['volume_usdb'] == 0) & (df['totalAmountPurchased'] > 0)]

,fromToken.decimals,fromToken.id,fromToken.symbol,toToken.decimals,toToken.id,toToken.symbol,totalAmountPurchased_start,totalAmountReturned_start,totalAmountPurchased_end,totalAmountReturned_end,totalAmountPurchased_start_f,totalAmountPurchased_end_f,totalAmountReturned_start_f,totalAmountReturned_end_f,totalAmountPurchased,totalAmountReturned,volume_bnt,volume_usdb
339,18.0,0x8a9c67fee641579deba04928c4bc45f66e26343a,JRT,18.0,0x4827e558e642861cd7a1c8f011b2b4661f8d51fa,JRTUSDB,67352857828516101799973,1241277474250029294520,68025840167416101343735,1259282664226745488444,67352.857829,68025.840167,1241.277474,1259.282664,672.982339,18.005190,0.0,0.0
387,18.0,0xb1cd6e4153b2a390cf00a6556b0fc1458c4a5533,ETHBNT,18.0,0xc0829421c1d260bd3cb3e0f06cfe2d52db2ce315,ETH,85017269735067518912635,152673453432967492107,86744931413371652736650,155705157045565593759,85017.269735,86744.931413,152.673453,155.705157,1727.661678,3.031704,0.0,0.0
399,18.0,0xbf2179859fc6d5bee9bf9158632dc51678a4100e,ELF,18.0,0x0f2318565f1996cb1ed2f88e172135791bc1fcbf,ELFBNT,3883000000000000000000,725547666992843469753,5183000000000000000000,1002058349917051189729,3883.000000,5183.000000,725.547667,1002.058350,1300.000000,276.510683,0.0,0.0
443,18.0,0xe6b31fb3f29fbde1b92794b0867a315ff605a324,BNBBNT,18.0,0xb8c77482e45f1f44de1745f52c74426c631bdd52,BNB,165737059105988105199,16782254058177895436,354916830208575403146,35631841703694962372,165.737059,354.916830,16.782254,35.631842,189.179771,18.849588,0.0,0.0
